#Set up the API

In [6]:
# Installing the required packages
%pip install -U openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [7]:
# Set environment variables and create openai client

import os
from openai import AzureOpenAI
from google.colab import userdata # To get the secret keys

# Deployment name in azure openai studio
gpt_model = "gpt-4o-mini"  # ex. gpt-4o-mini

client = AzureOpenAI(
    api_key=userdata.get('AZURE_OPENAI_API_KEY'),
    api_version="2023-03-15-preview",
    azure_endpoint=userdata.get('AZURE_OPENAI_ENDPOINT'),
)

In [8]:
# Test that everything is working
messages = [{"role": "user", "content": "Hello, ready for some AI analysis?"}]
response = client.chat.completions.create(model=gpt_model, messages=messages, max_tokens=50)

print("Response: \n", response.choices[0].message.content)

# Expected respnse should be something like:
# Response:
#  Absolutely! What do you need analyzed?

Response: 
 Hello! Absolutely, I'm ready for some AI analysis. What topic or area would you like to explore?


In [ ]:
!pip install yt-dlp

import yt_dlp
import os

##---------------------------------------Test for subtitles download**


def main():
    url = "https://tv.nrk.no/serie/debatten/sesong/202410/episode/NNFA51101024"

    download_path = os.path.join(os.getcwd(), "Debatten_subtitles")
    os.makedirs(download_path, exist_ok=True)

    ydl_opts = {
        'writesubtitles': True,
        'subtitleslangs': ['nb-ttv'],  # Norsk bokmål TV-teksting
        'skip_download': True,  # Hopp over nedlasting av video
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
        'quiet': False,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegSubtitlesConvertor',
            'format': 'srt',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"Subtitles downloaded to -> : {download_path}")
    except Exception as e:
        print(f"En feil oppstod: {e}")

if __name__ == "__main__":
    main()

  Using cached yt_dlp-2024.11.4-py3-none-any.whl.metadata (172 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.5 MB/s eta 0:00:00
[NRKTV] Extracting URL: https://tv.nrk.no/serie/debatten/sesong/202410/episode/NNFA51101024
[NRK] Extracting URL: nrk:NNFA51101024
[NRK] NNFA51101024: Downloading manifest JSON
[NRK] NNFA51101024: Downloading m3u8 information
[NRK] NNFA51101024: Downloading metadata JSON
[NRK] NNFA51101024: Downloading programs JSON
[info] NNFA51101024: Downloading subtitles: nb-ttv
[info] NNFA51101024: Downloading 1 format(s): 2350+aud2-Unknown
[info] Writing video subtitles to: /content/Debatten_subtitles/Debatten - 10. okt. · Bløffebudsjett, mener opposisjonen.nb-ttv.vtt
[download] Destination: /content/Debatten_subtitles/Debatten - 10. okt. · Bløffebudsjett, mener opposisjonen.nb-ttv.vtt
[download] 100% of   63.29KiB in 00:00:00 at 87.56KiB/s  
Subtitles downloaded to -> : /content/Debatten_subtitles


#Food-debate


In [ ]:
import json
from openai.types.chat import ChatCompletionUserMessageParam, ChatCompletionSystemMessageParam

path_to_file = "Test.txt"

# Read the data from the file
with open(path_to_file, "r", encoding="UTF-8") as file:
    data = file.read()

messages = [
    ChatCompletionSystemMessageParam(
        role="system",
        content="""
            You are an expert in language analysis and interaction analysis in discussions, with a focus on identifying all unique speakers in dialogue.
Your task is to analyze the entire text provided and:

1. Identify and label each unique speaker based on their speech patterns, contextual cues, or dialogue markers.
2. Assign a unique identifier to each speaker (e.g., "Speaker 1," "Speaker 2," etc.) and determine their gender (M/F) when possible based on context.
3. Include all participants present in the text and ensure no speaker is omitted. Do not limit the identification to a predefined number of examples.
4. Look for shifts in tone, specific language usage, or structural cues in the dialogue to differentiate speakers.
3. Examine Interaction Styles:
            - Detailed Tone Categorization: Break down the tone analysis into subcategories, such as empathetic, argumentative, defensive, or conciliatory, to gain a more comprehensive understanding of the interaction style.
            - Emotion Detection: Incorporate sentiment analysis tools to detect emotions like frustration, enthusiasm, or sarcasm. This helps capture subtle nuances in interactions that extend beyond simple supportive or critical categories.
            - Power Dynamics: Analyze indications of power imbalances, such as frequent interruptions, speaking over others, or dismissive responses.
4. Use an analysis model: Analyze how gender patterns may influence treatment in interactions, while controlling for other variables.

Analyze the interactions between all participants and categorize them into gender-based combinations (M to M, F to M, F to F, M to F). Ensure the output is presented in JSON format as follows:
{
    "episode_name": "<generated_name>",
    "participants": [
        {
            "role": "<role>",
            "gender": "<M/F>",
            "background": "(party affiliation, organization, etc.)"
        }
    ],
    "interaction_analysis": {
        "M_to_M": {
            "summary": "<summary>",
            "score": <number>
        },
        "F_to_F": {
            "summary": "<summary>",
            "score": <number>
        },
        "F_to_M": {
            "summary": "<summary>",
            "score": <number>
        },
        "M_to_F": {
            "summary": "<summary>",
            "score": <number>
        }
    },
    "overall_trends": "<description>",
    "final_score": <number>
}
""",
    ),
    ChatCompletionUserMessageParam(
        role="user",
        content=f"""
            Analyze the text below and categorize participants as “debater 1,” “debater 2,” “expert 1,” etc.
            Evaluate how they interact with each other in all gender combinations (M to M, F to M, F to F, M to F).
            Include all participants and do not include the moderator in the analysis.
            In English only.

            <data>{data}</data>

        Response:
        """,
    ),
]

completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Replace with your actual model name
    messages=messages,
    temperature=0,
    response_format={"type": "json_object"},  # Guarantees that the response is in JSON format
)

assert completion.choices[0].message.content is not None

# Print and parse the JSON response
print(completion.choices[0].message.content)

analysis_result = json.loads(completion.choices[0].message.content)
print("Overall score for analysis:", analysis_result["final_score"])


#Political Debate


In [ ]:
import json
from openai.types.chat import ChatCompletionUserMessageParam, ChatCompletionSystemMessageParam

path_to_file = "/content/Debatten_subtitles/Test.vtt"

# Read the data from the file
with open(path_to_file, "r", encoding="UTF-8") as file:
    data = file.read()

messages = [
    ChatCompletionSystemMessageParam(
        role="system",
        content="""
            You are an expert in language analysis and interaction analysis in discussions, with a focus on identifying all unique speakers in dialogue.
Your task is to analyze the entire text provided and:

1. Identify and label each unique speaker based on their speech patterns, contextual cues, or dialogue markers.
2. Assign a unique identifier to each speaker (e.g., "Speaker 1," "Speaker 2," etc.) and determine their gender (M/F) when possible based on context.
3. Include all participants present in the text and ensure no speaker is omitted. Do not limit the identification to a predefined number of examples.
4. Look for shifts in tone, specific language usage, or structural cues in the dialogue to differentiate speakers.
5. Comprehensive Interaction Analysis: Evaluate and score the interactions between all debaters, not just a sample. This analysis should include every interaction and response in the provided text to ensure an accurate and holistic assessment of how each participant interacts with others.
  Scoring Categories:
    -Argument Strength: Rate the clarity, relevance, and persuasiveness of each participant’s arguments throughout the discussion.
    -Engagement Style: Assess interaction qualities such as empathy, assertiveness, defensiveness, or conciliation as demonstrated across all dialogue.
    -Tone Dynamics: Identify variations in tone such as supportive, critical, sarcastic, or enthusiastic exchanges, considering the entire range of interactions.
    -Power Dynamics: Score power displays or imbalances, such as interruptions, speaking over others, or dismissive responses, over the entire debate.
6. Use an analysis model: Analyze how gender patterns may influence treatment in interactions, while controlling for other variables.
7. Scoring: All scoring should be in the interval 1-10, 1 is the lowest and 10 is the highest.
8. Final Scoring: Combine scores across all categories for each participant to assign a final score reflecting their overall performance and engagement style in the full debate.

Analyze the interactions between all participants and categorize them into gender-based combinations (M to M, F to M, F to F, M to F). Ensure the output is presented in JSON format as follows:
{
    "episode_name": "<generated_name>",
    "participants": [
        {
            "role": "<role>",
            "gender": "<M/F>",
            "background": "(party affiliation, organization, etc.)"
        }
    ],
                        "interaction_analysis": {
                            "M_to_M": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"

                            },
                            "F_to_F": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            },
                            "F_to_M": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            },
                            "M_to_F": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            }
                        },
                        "overall_trends": "<description>",
                        "final_score": <number>
                            }
""",
    ),
    ChatCompletionUserMessageParam(
        role="user",
        content=f"""
            Analyze the text below and categorize participants as “debater 1,” “debater 2,” “expert 1,” etc not their real name.
            Evaluate how they interact with each other in all gender combinations (M to M, F to M, F to F, M to F).
            Include all participants and do not include the moderator in the analysis.
            In English only.

            <data>{data}</data>

        Response:
        """,
    ),
]

completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Replace with your actual model name
    messages=messages,
    temperature=0.5,
    response_format={"type": "json_object"},  # Guarantees that the response is in JSON format
)

assert completion.choices[0].message.content is not None

# Print and parse the JSON response
print(completion.choices[0].message.content)

analysis_result = json.loads(completion.choices[0].message.content)
print("Overall score for analysis:", analysis_result["final_score"])


{
    "episode_name": "Norwegian Political Debate Analysis",
    "participants": [
        {
            "role": "Debater 1",
            "gender": "M",
            "background": "Leader of the Labour Party (Ap)"
        },
        {
            "role": "Debater 2",
            "gender": "F",
            "background": "Leader of the Conservative Party (Høyre)"
        },
        {
            "role": "Debater 3",
            "gender": "M",
            "background": "Leader of the Progress Party (Frp)"
        },
        {
            "role": "Debater 4",
            "gender": "F",
            "background": "Leader of the Socialist Left Party (SV)"
        },
        {
            "role": "Debater 5",
            "gender": "F",
            "background": "Leader of the Centre Party (Sp)"
        },
        {
            "role": "Debater 6",
            "gender": "F",
            "background": "Leader of the Red Party (Rødt)"
        },
        {
            "role": "Debater 7",
         

In [21]:
import pandas as pd

# Assuming analysis_result is already defined
episode_name = analysis_result['episode_name']

# Create scores_data DataFrame
scores_data = []
for interaction, details in analysis_result["interaction_analysis"].items():
    scores_data.append({
        "episode_name": episode_name,
        "interaction_type": interaction,
        "argument_strength": details["Argument Strength"],
        "summary_argument_strength": details["Summary Argument Strength"],
        "engagement_style": details["Engagement Style"],
        "summary_engagement_style": details["Summary Engagement Style"],
        "tone_dynamics": details["Tone Dynamics"],
        "summary_tone_dynamics": details["Summary Tone Dynamics"],
        "power_dynamics": details["Power Dynamics"],
        "summary_power_dynamics": details["Summary Power Dynamics"],
        "total_score": details["Total score (average of all)"],
        "total_summary": details["Total summary"]
    })

scores_df = pd.DataFrame(scores_data)

# Create participants_data DataFrame
participants_data = []
for participant in analysis_result["participants"]:
    participants_data.append({
        "episode_name": episode_name,
        "role": participant["role"],
        "gender": participant["gender"],
        "background": participant["background"]
    })

participants_df = pd.DataFrame(participants_data)

# Display the DataFrames
print("Scores DataFrame:")
print(scores_df)
print("\nParticipants DataFrame:")
print(participants_df)

Scores DataFrame:
                                        episode_name interaction_type  \
0  Debatten - 8. okt. · Tidenes budsjett, men mis...           M_to_M   
1  Debatten - 8. okt. · Tidenes budsjett, men mis...           F_to_F   
2  Debatten - 8. okt. · Tidenes budsjett, men mis...           F_to_M   
3  Debatten - 8. okt. · Tidenes budsjett, men mis...           M_to_F   

   argument_strength                          summary_argument_strength  \
0                  7  Both male debaters presented strong arguments ...   
1                  8  The female experts provided insightful perspec...   
2                  9  Female participants effectively challenged mal...   
3                  7  Male participants presented strong arguments b...   

   engagement_style                           summary_engagement_style  \
0                 6  Engagement was generally assertive, with some ...   
1                 7  Engagement was collaborative, with a focus on ...   
2                 

In [ ]:
# Save the DataFrames as separate Excel files in /content/ directory
scores_output_path = '/content/scores_analysis_results.xlsx'
participants_output_path = '/content/participants_analysis_results.xlsx'

# Save each DataFrame to an Excel file
scores_df.to_excel(scores_output_path, index=False)
participants_df.to_excel(participants_output_path, index=False)

(scores_output_path, participants_output_path)


('/content/scores_analysis_results.xlsx',
 '/content/participants_analysis_results.xlsx')

In [ ]:
# Lag en matrise med poeng for interaksjoner der radene og kolonnene representerer kjønn (M og F)
gender_matrix = pd.DataFrame(index=["M", "F"], columns=["M", "F"])

# Fyll ut matrisen basert på interaksjonstype i DataFrame
for _, row in df_interaction_analysis.iterrows():
    if row['interaction_type'] == "M_to_M":
        gender_matrix.at["M", "M"] = row['score']
    elif row['interaction_type'] == "F_to_F":
        gender_matrix.at["F", "F"] = row['score']
    elif row['interaction_type'] == "F_to_M":
        gender_matrix.at["F", "M"] = row['score']
    elif row['interaction_type'] == "M_to_F":
        gender_matrix.at["M", "F"] = row['score']

# Vis matrisen
gender_matrix


,M,F
M,7,7
F,6,8


In [3]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.2 MB/s eta 0:00:00


#Bigdata

In [4]:
import requests
import yt_dlp
import os

# Base URL for NRK's API
base_url = "https://psapi.nrk.no"
series_url = f"{base_url}/tv/catalog/series/debatten"

# Antall episoder vi ønsker å hente, justert for å ignorere første
target_episodes = 9
episodes_needed = target_episodes + 1  # Hent 10 episoder, men ignorer den første
episodes_fetched = 0
page = 1

try:
    all_episodes = []

    # Hent episoder fra API-et
    while len(all_episodes) < episodes_needed:
        response = requests.get(series_url, params={"page": page})
        response.raise_for_status()
        series_data = response.json()

        if '_embedded' in series_data and 'instalments' in series_data['_embedded']:
            instalment_data = series_data['_embedded']['instalments']
            if '_embedded' in instalment_data and 'instalments' in instalment_data['_embedded']:
                episodes = instalment_data['_embedded']['instalments']
                all_episodes.extend(episodes)

                # Sjekk om det er flere sider
                if '_links' in instalment_data and 'next' in instalment_data['_links']:
                    page += 1
                else:
                    break  # Ingen flere sider
            else:
                break  # Ingen flere episoder funnet
        else:
            break  # Ingen flere episoder funnet

    # Fjern den første episoden og samle URL-ene for resten
    all_episodes = all_episodes[1:episodes_needed]  # Ignorer den første episoden

    # Samle URL-ene i en liste kalt URL
    URL = [f'https://tv.nrk.no/se?v={episode["prfId"]}' for episode in all_episodes]

    # Print en melding som viser hvor mange episoder som er hentet
    print(f"Episodes of Debatten (totalt {len(URL)}):")

    # Funksjon for å laste ned undertekster
    def download_subtitles(url, download_path):
        ydl_opts = {
            'writesubtitles': True,
            'subtitleslangs': ['nb-ttv'],  # Norsk bokmål TV-teksting
            'skip_download': True,  # Hopp over nedlasting av video
            'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),
            'quiet': True,  # Reduserer mengden output til et minimum
            'no_warnings': True,  # Skjuler advarsler
            'postprocessors': [{
                'key': 'FFmpegSubtitlesConvertor',
                'format': 'srt',
            }],
            'logger': None,  # Fjerner logger-objektet for mindre støy
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            print(f"Subtitles downloaded for URL: {url}")
            return True  # Returner True hvis nedlasting var vellykket
        except Exception as e:
            print(f"En feil oppstod for {url}: {e}")
            return False  # Returner False hvis det oppstod en feil

    # Opprett mappe for lagring av undertekster
    download_path = os.path.join(os.getcwd(), "Debatten_subtitles")
    os.makedirs(download_path, exist_ok=True)

    # Last ned undertekster for hver URL i listen
    valid_urls = []
    for i, url in enumerate(URL):
        success = download_subtitles(url, download_path)
        if not success:
            print("Prøver neste tilgjengelige episode...")
            continue  # Gå videre til neste episode
        valid_urls.append(url)

    if len(valid_urls) < target_episodes:
        print(f"Kun {len(valid_urls)} episoder ble lastet ned vellykket.")
    else:
        print("All episodes fetched sucsessfully.")

except requests.RequestException as e:
    print(f"En feil oppstod ved henting av data: {e}")


Episodes of Debatten (totalt 9):
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51110724
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51103124
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51102924
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51102424
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51102224
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51101724
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51101524
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51101024
Subtitles downloaded for URL: https://tv.nrk.no/se?v=NNFA51100824
All episodes fetched sucsessfully.


In [20]:
import os
import json
import openai


# Katalog som inneholder VTT-filer
directory_path = "Debatten_subtitles"

# Liste for å lagre analyseresultater for alle filer
analysis_results = []

# Iterer gjennom alle VTT-filer i katalogen
for filename in os.listdir(directory_path):
    if filename.endswith(".vtt"):
        file_path = os.path.join(directory_path, filename)

        # Les data fra VTT-filen
        with open(file_path, "r", encoding="UTF-8") as file:
            data = file.read()

        # Opprett meldingene for forespørselen
        messages = [
            {
                "role": "system",
                "content": """
                    You are an expert in language analysis and interaction analysis in discussions, with a focus on identifying all unique speakers in dialogue.
                    Your task is to analyze the entire text provided and:

                    1. Identify and label each unique speaker based on their speech patterns, contextual cues, or dialogue markers.
                    2. Assign a unique identifier to each speaker (e.g., "Speaker 1," "Speaker 2,", "Debater 1", "Debater 2," or "Expert 1" etc.) and determine their gender (M/F) when possible based on context. Not their real name.
                    3. Include all participants present in the text and ensure no speaker is omitted. Do not limit the identification to a predefined number of examples.
                    4. Look for shifts in tone, specific language usage, or structural cues in the dialogue to differentiate speakers.
                    5. Comprehensive Interaction Analysis: Evaluate and score the interactions between all debaters, not just a sample. This analysis should include every interaction and response in the provided text to ensure an accurate and holistic assessment of how each participant interacts with others.
                      Scoring Categories:
                        -Argument Strength: Rate the clarity, relevance, and persuasiveness of each participant’s arguments throughout the discussion.
                        -Engagement Style: Assess interaction qualities such as empathy, assertiveness, defensiveness, or conciliation as demonstrated across all dialogue.
                        -Tone Dynamics: Identify variations in tone such as supportive, critical, sarcastic, or enthusiastic exchanges, considering the entire range of interactions.
                        -Power Dynamics: Score power displays or imbalances, such as interruptions, speaking over others, or dismissive responses, over the entire debate.
                    6. Use an analysis model: Analyze how gender patterns may influence treatment in interactions, while controlling for other variables.
                    7. Scoring: All scoring should be in the interval 1-10, 1 is the lowest and 10 is the highest.
                    8. Final Scoring: Combine scores across all categories for each participant to assign a final score reflecting their overall performance and engagement style in the full debate.

                    Analyze the interactions between all participants and categorize them into gender-based combinations (M to M, F to M, F to F, M to F). Ensure the output is presented in JSON format as follows:
                    {
                        "episode_name": "<generated_name>",
                        "participants": [
                            {
                                "role": "<role>",
                                "gender": "<M/F>",
                                "background": "(party affiliation, organization, etc.)"
                            }
                        ],
                        "interaction_analysis": {
                            "M_to_M": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"

                            },
                            "F_to_F": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            },
                            "F_to_M": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            },
                            "M_to_F": {
                                "Argument Strenght": <number>,
                                "Summary Argument Strenght": <summary>,
                                "Engagement Style": <number>,
                                "Summary Engagement style": <summary>,
                                "Tone Dynamics": <number>,
                                "Summary Tone Dynamics": <summary>,
                                "Power Dynamics": <number>,
                                "Summary Power Dynamics": <summary>,
                                "Total score (average of all)": <number>,
                                "Total summary": "<summary>"
                            }
                        },
                        "overall_trends": "<description>",
                        "final_score": <number>
                    }
                """
            },
            {
                "role": "user",
                "content": f"""
                    Analyze the text below and categorize participants as “debater 1,” “debater 2,” “expert 1,” etc not their real name.
                    Evaluate how they interact with each other in all gender combinations (M to M, F to M, F to F, M to F).
                    Include all participants and do not include the moderator in the analysis.
                    In English only.

                    <data>{data}</data>

                    Response:
                """
            }
        ]

        # Forespørsel til OpenAI API
        completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Replace with your actual model name
        messages=messages,
        temperature=0.5,
        response_format={"type": "json_object"},  # Guarantees that the response is in JSON format
)

        # Analyser og lagre resultatet
        if completion.choices[0].message.content:
            analysis_result = json.loads(completion.choices[0].message.content)
            analysis_result["episode_name"] = filename.replace(".vtt", "")  # Legg til episodenavn
            analysis_results.append(analysis_result)

            # Print melding når analysen er ferdig for en episode
            print(f"Finished analyzing: {filename}")

# Print den endelige JSON-variabelen som inneholder alle resultater
final_json_result = json.dumps(analysis_results, indent=4)
print(final_json_result)


Finished analyzing: Debatten - 10. okt. · Bløffebudsjett, mener opposisjonen.nb-ttv.vtt
Finished analyzing: Debatten - 22. okt. · Milliardene flyr i havvinden.nb-ttv.vtt
Finished analyzing: Debatten - 15. okt. · Skoler legges ned, bygder dør.nb-ttv.vtt
Finished analyzing: Debatten - Torsdag · «Falsk same»？.nb-ttv.vtt
Finished analyzing: Debatten - 29. okt. · Du blir lurt.nb-ttv.vtt
Finished analyzing: Debatten - 24. okt. · Unge sender Ap mot sperregrensa.nb-ttv.vtt
Finished analyzing: Debatten - 31. okt. · Milliarder til norske batterier.nb-ttv.vtt
Finished analyzing: Debatten - 17. okt. · Slik kan du spare på maten.nb-ttv.vtt
Finished analyzing: Debatten - 8. okt. · Tidenes budsjett, men misnøyen vokser.nb-ttv.vtt
[
    {
        "episode_name": "Debatten - 10. okt. \u00b7 Bl\u00f8ffebudsjett, mener opposisjonen.nb-ttv",
        "participants": [
            {
                "role": "Debater 1",
                "gender": "M",
                "background": "Prime Minister, Labour Part

##Make to DF

In [16]:
import pandas as pd

# Assuming final_json_result is already loaded
# final_json_result = json.loads(json_string)

# Create scores_data DataFrame
scores_data = []
for episode in final_json_result:
    episode_name = episode['episode_name']
    for interaction, details in episode["interaction_analysis"].items():
        scores_data.append({
            "episode_name": episode_name,
            "interaction_type": interaction,
            "argument_strength": details.get("Argument Strength", details.get("Argument Strenght", 0)),
            "summary_argument_strength": details.get("Summary Argument Strength", details.get("Summary Argument Strenght", "")),
            "engagement_style": details.get("Engagement Style", 0),
            "summary_engagement_style": details.get("Summary Engagement Style", details.get("Summary Engagement style", "")),
            "tone_dynamics": details.get("Tone Dynamics", 0),
            "summary_tone_dynamics": details.get("Summary Tone Dynamics", ""),
            "power_dynamics": details.get("Power Dynamics", 0),
            "summary_power_dynamics": details.get("Summary Power Dynamics", ""),
            "total_score": details.get("Total score (average of all)", 0),
            "total_summary": details.get("Total summary", "")
        })

scores_df = pd.DataFrame(scores_data)

# Create participants_data DataFrame
participants_data = []
for episode in final_json_result:
    episode_name = episode['episode_name']
    for participant in episode.get("participants", []):
        participants_data.append({
            "episode_name": episode_name,
            "role": participant.get("role", ""),
            "gender": participant.get("gender", ""),
            "background": participant.get("background", "")
        })

participants_df = pd.DataFrame(participants_data)

# Display the DataFrames
print("Scores DataFrame:")
print(scores_df)
print("\nParticipants DataFrame:")
print(participants_df)

Scores DataFrame:
                                         episode_name interaction_type  \
0   Debatten - 10. okt. · Bløffebudsjett, mener op...           M_to_M   
1   Debatten - 10. okt. · Bløffebudsjett, mener op...           F_to_F   
2   Debatten - 10. okt. · Bløffebudsjett, mener op...           F_to_M   
3   Debatten - 10. okt. · Bløffebudsjett, mener op...           M_to_F   
4   Debatten - 22. okt. · Milliardene flyr i havvi...           M_to_M   
5   Debatten - 22. okt. · Milliardene flyr i havvi...           F_to_F   
6   Debatten - 22. okt. · Milliardene flyr i havvi...           F_to_M   
7   Debatten - 22. okt. · Milliardene flyr i havvi...           M_to_F   
8   Debatten - 15. okt. · Skoler legges ned, bygde...           M_to_M   
9   Debatten - 15. okt. · Skoler legges ned, bygde...           F_to_F   
10  Debatten - 15. okt. · Skoler legges ned, bygde...           F_to_M   
11  Debatten - 15. okt. · Skoler legges ned, bygde...           M_to_F   
12          Debatten

##Save as xlsx

In [17]:
# Save the DataFrames as separate Excel files in /content/ directory
scores_output_path = '/content/scores_analysis_results.xlsx'
participants_output_path = '/content/participants_analysis_results.xlsx'

# Save each DataFrame to an Excel file
scores_df.to_excel(scores_output_path, index=False)
participants_df.to_excel(participants_output_path, index=False)

(scores_output_path, participants_output_path)


('/content/scores_analysis_results.xlsx',
 '/content/participants_analysis_results.xlsx')

In [19]:
import pandas as pd
import json

# Assuming final_json_result is already loaded
# final_json_result = json.loads(json_string)

# Create scores_data DataFrame
scores_data = []
for episode in final_json_result:
    episode_name = episode['episode_name']
    for interaction_type, details in episode["interaction_analysis"].items():
        scores_data.append({
            "episode_name": episode_name,
            "interaction_type": interaction_type,
            "Argument Strength": details.get("Argument Strength", details.get("Argument Strenght", 0)),
            "Engagement Style": details.get("Engagement Style", 0),
            "Tone Dynamics": details.get("Tone Dynamics", 0),
            "Power Dynamics": details.get("Power Dynamics", 0),
            "Total Score": details.get("Total score (average of all)", 0)
        })

scores_df = pd.DataFrame(scores_data)

# Group by interaction_type and calculate average scores
average_scores = scores_df.groupby('interaction_type').agg({
    'Argument Strength': 'mean',
    'Engagement Style': 'mean',
    'Tone Dynamics': 'mean',
    'Power Dynamics': 'mean',
    'Total Score': 'mean'
}).round(2)  # Round to 2 decimal places for readability

# Calculate overall average across all interaction types
overall_average = average_scores.mean().round(2)
overall_average.name = 'Overall Average'

# Add overall average to the results using concat instead of append
final_results = pd.concat([average_scores, overall_average.to_frame().T])

print("Average Scores by Interaction Type:")
print(final_results)

# Optionally, you can save this to a CSV file
final_results.to_csv('average_scores_by_interaction.csv')

Average Scores by Interaction Type:
                 Argument Strength  Engagement Style  Tone Dynamics  \
F_to_F                        7.33              7.00           6.56   
F_to_M                        6.89              6.22           5.89   
M_to_F                        6.11              5.22           5.33   
M_to_M                        7.33              6.33           6.11   
Overall Average               6.91              6.19           5.97   

                 Power Dynamics  Total Score  
F_to_F                     5.33         6.53  
F_to_M                     5.44         6.17  
M_to_F                     5.67         5.59  
M_to_M                     6.00         6.49  
Overall Average            5.61         6.20  
